In [1]:
### Load packages and csv data

using JuMP, HiGHS, CSV, DataFrames, LinearAlgebra

distances_data = CSV.read("distances-v3-oldfunkyformat.csv",DataFrame,delim=',')
loads_data = CSV.read("loads-v3.csv",DataFrame,delim=',')
trucks_data = CSV.read("trucks-v3.csv",DataFrame,delim=',')
endloccount_data = CSV.read("endloccount-v3.csv",DataFrame,delim=',')

Row,Column1
,Int64
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [2]:
### Sets

# Set of locations
N = nrow(endloccount_data)

# Set of loads
K = nrow(loads_data)

10

In [4]:
### Data

# Distance between location i, j
Dis = zeros((nrow(endloccount_data),nrow(endloccount_data)))

for k in 1:nrow(endloccount_data)
    Dis[k:k,1:end] = parse.(Int, split(chop(distances_data[k,1]; head=1, tail=1), ','))
end

# Cost for traveling between location i, j
C = Dis*0.8

# Number of time periods
T_4hr = 42
T_1hr = 168
T_15min = 672

# Origin and destination of load k
O = loads_data[1:end,1]
Des = loads_data[1:end,2]

# Number of trucks newly available at location i in period t
A_4hr_cons = zeros(Int, (nrow(endloccount_data),T_4hr))
A_4hr_opt = zeros(Int, (nrow(endloccount_data),T_4hr))

A_1hr_cons = zeros(Int, (nrow(endloccount_data),T_1hr))
A_1hr_opt = zeros(Int, (nrow(endloccount_data),T_1hr))

A_15min_cons = zeros(Int, (nrow(endloccount_data),T_15min))
A_15min_opt = zeros(Int, (nrow(endloccount_data),T_15min))

trucks_4hr_cons = CSV.read("trucks-v3.csv",DataFrame,delim=',')
trucks_4hr_cons[1:end,1:1] = ceil.(Int, trucks_4hr_cons[1:end,1:1].*0.25)
trucks_4hr_cons = round.(Int,trucks_4hr_cons)
trucks_4hr_opt = CSV.read("trucks-v3.csv",DataFrame,delim=',')
trucks_4hr_opt[1:end,1:1] = ceil.(Int, trucks_4hr_opt[1:end,1:1].*0.25)
trucks_4hr_opt = round.(Int,trucks_4hr_opt)

trucks_1hr_cons = ceil.(Int, trucks_data)
trucks_1hr_opt = floor.(Int, trucks_data)

trucks_15min_cons = CSV.read("trucks-v3.csv",DataFrame,delim=',')
trucks_15min_cons[1:end,1:1] = ceil.(Int, trucks_15min_cons[1:end,1:1].*4)
trucks_15min_cons = round.(Int,trucks_15min_cons)
trucks_15min_opt = CSV.read("trucks-v3.csv",DataFrame,delim=',')
trucks_15min_opt[1:end,1:1] = floor.(Int, trucks_15min_cons[1:end,1:1].*4)
trucks_15min_opt = round.(Int,trucks_15min_opt)

for i in 1:nrow(trucks_data)
    A_4hr_cons[trucks_4hr_cons[i,2],trucks_4hr_cons[i,1]] = A_4hr_cons[trucks_4hr_cons[i,2],trucks_4hr_cons[i,1]] + 1
    A_4hr_opt[trucks_4hr_opt[i,2],trucks_4hr_opt[i,1]] = A_4hr_opt[trucks_4hr_opt[i,2],trucks_4hr_opt[i,1]] + 1
    A_1hr_cons[trucks_1hr_cons[i,2],trucks_1hr_cons[i,1]] = A_1hr_cons[trucks_1hr_cons[i,2],trucks_1hr_cons[i,1]] + 1
    A_1hr_opt[trucks_1hr_opt[i,2],trucks_1hr_opt[i,1]] = A_1hr_opt[trucks_1hr_opt[i,2],trucks_1hr_opt[i,1]] + 1
    A_15min_cons[trucks_15min_cons[i,2],trucks_15min_cons[i,1]] = A_15min_cons[trucks_15min_cons[i,2],trucks_15min_cons[i,1]] + 1
    A_15min_opt[trucks_15min_opt[i,2],trucks_15min_opt[i,1]] = A_15min_opt[trucks_15min_opt[i,2],trucks_15min_opt[i,1]] + 1    
end

# Earliest and Latest pickup time(in hours) of load k
S = loads_data[1:end,3]
L = loads_data[1:end,4]

for i in 1:nrow(loads_data)
    if L[i] >= T_1hr + 1
        L[i] = T_1hr
    end
end

S_4hr_cons = ceil.(Int, S.*0.25)
L_4hr_cons = floor.(Int, L.*0.25)

S_4hr_opt = floor.(Int, S.*0.25)
L_4hr_opt = ceil.(Int, L.*0.25)

S_1hr_cons = ceil.(Int, S)
L_1hr_cons = floor.(Int, L)

S_1hr_opt = floor.(Int, S)
L_1hr_opt = ceil.(Int, L)

S_15min_cons = ceil.(Int, S.*4)
L_15min_cons = floor.(Int, L.*4)

S_15min_opt = floor.(Int, S.*4)
L_15min_opt = ceil.(Int, L.*4)

# Revenue for carrying load k
R = loads_data[1:end,5]

# Number of trucks desired at location i
B = endloccount_data[1:end,1]

# Time to travel from location i to location j
Tau = Dis/60

Tau_4hr_cons = ceil.(Int, Tau.*0.25)
Tau_4hr_cons += diagm(ones(Int, 22))
Tau_4hr_opt = floor.(Int, Tau.*0.25)
Tau_4hr_opt += diagm(ones(Int, 22))

Tau_1hr_cons = ceil.(Int, Tau)
Tau_1hr_cons += diagm(ones(Int, 22))
Tau_1hr_opt = floor.(Int, Tau)
Tau_1hr_opt += diagm(ones(Int, 22))

Tau_15min_cons = ceil.(Int, Tau.*4)
Tau_15min_cons += diagm(ones(Int, 22))
Tau_15min_opt = floor.(Int, Tau.*4)
Tau_15min_opt += diagm(ones(Int, 22))

22×22 Matrix{Int64}:
  1  40  55  41  43   92  67  26   22  …  65  13   78  17  51  20  34  51  27
 40   1  32   3  54   54  30  25   59     25  49   48  53  67  20  74  14  19
 55  32   1  34  21   36  11  28   78     57  42   22  38  35  39  41  20  37
 41   3  34   1  56   58  33  27   56     23  50   50  54  69  21  76  17  15
 43  54  21  56   1   48  29  28   65     79  30   35  26  13  35  28  42  42
 92  54  36  58  48    1  24  65  114  …  53  78   20  74  40  75  76  40  73
 67  30  11  33  29   24   1  40   89     50  54   23  50  42  50  51  15  49
 26  25  28  27  28   65  40   1   49     50  23   51  27  41  10  48  24  13
 22  59  78  56  65  114  89  49    1     66  35  100  39  73  38  37  73  40
 35  71  39  73  17   56  46  46   57     96  22   43  18  16  52  19  59  59
 12  52  53  53  41   89  65  26   24  …  77  10   76  14  48  32  22  49  40
 38  50  17  51   5   53  29  24   60     75  25   40  21  17  30  24  38  38
 20  52  35  53  23   71  46  26   42     7

In [5]:
### Optimization model for 1 hour, conservative approximation

# Create the model
m_1hr_cons = Model()

# Declare all the decision variables
@variable(m_1hr_cons, x[1:N,1:N,1:T_1hr] >= 0, Int)

@variable(m_1hr_cons, y[1:K], Bin)

# Define the objective using expression
@expression(m_1hr_cons, objective_expr,
    sum(R[k]y[k] for k in 1:K) - sum(C[i,j]x[i,j,t] for i in 1:N, j in 1:N, t in 1:T_1hr-Tau_1hr_cons[i,j]))

@objective(m_1hr_cons, Max, objective_expr)

# Write all the constraints
@constraint(m_1hr_cons, trucks_flow[i in 1:N, t in 1:T_1hr-1],
        A_1hr_cons[i,t] + sum(x[j,i,t-Tau_1hr_cons[j,i]] for j in 1:N if t-Tau_1hr_cons[j,i] >= 1) == sum(x[i,j,t] for j in 1:N))
    
@constraint(m_1hr_cons, trucks_end[i in 1:N], 
    sum(x[j,i,T_1hr-Tau_1hr_cons[j,i]] for j in 1:N if T_1hr-Tau_1hr_cons[j,i] >= 1) == B[i])

@constraint(m_1hr_cons, loads_carry[k in 1:K], 
    sum(x[O[k],Des[k],t] for t in S_1hr_cons[k]:L_1hr_cons[k]) >= y[k])

# Solve model
set_optimizer(m_1hr_cons, HiGHS.Optimizer)
set_time_limit_sec(m_1hr_cons, 1800)

optimize!(m_1hr_cons)

# Print total profit
println("Total profit: ", objective_value(m_1hr_cons))

# Specific movement of trucks
for t in 1:T_1hr, i in 1:N, j in 1:N 
    if value(x[i,j,t]) > 0.9999
    println("Truck moved from ", i, " to ", j, " at time ", t)
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3362 rows, 73291 cols, 142089 nonzeros
3156 rows, 65570 cols, 130553 nonzeros
Objective function is integral with scale 5

Solving MIP model with:
   3156 rows
   65570 cols (1232 binary, 64338 integer, 0 implied int., 0 continuous)
   130553 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   14968           -inf                 inf        0      0      0         0     2.6s
         0       0         0   0.00%   8419.8          -inf                 inf        0      0      4      3136   406.4s
         0       0         0   0.00%   8114.6          -inf                 inf      178      6      6      3647   890.2s
         0       0         0   0.00%   8114.6    

In [ ]:
### Optimization model for 1 hour, optimistical approximation

# Create the model
m_1hr_opt = Model()

# Declare all the decision variables
@variable(m_1hr_opt, x[1:N,1:N,1:T_1hr] >= 0, Int)

@variable(m_1hr_opt, y[1:K], Bin)

# Define the objective using expression
@expression(m_1hr_opt, objective_expr,
    sum(R[k]y[k] for k in 1:K) - sum(C[i,j]x[i,j,t] for i in 1:N, j in 1:N, t in 1:T_1hr-Tau_1hr_opt[i,j]))

@objective(m_1hr_opt, Max, objective_expr)

# Write all the constraints
@constraint(m_1hr_opt, trucks_flow[i in 1:N, t in 1:T_1hr-1],
        A_1hr_opt[i,t] + sum(x[j,i,t-Tau_1hr_opt[j,i]] for j in 1:N if t-Tau_1hr_opt[j,i] >= 1) == sum(x[i,j,t] for j in 1:N))
    
@constraint(m_1hr_opt, trucks_end[i in 1:N], 
    sum(x[j,i,T_1hr-Tau_1hr_opt[j,i]] for j in 1:N if T_1hr-Tau_1hr_opt[j,i] >= 1) == B[i])

@constraint(m_1hr_opt, loads_carry[k in 1:K], 
    sum(x[O[k],Des[k],t] for t in S_1hr_opt[k]:L_1hr_opt[k]) >= y[k])

# Solve model
set_optimizer(m_1hr_opt, HiGHS.Optimizer)
set_time_limit_sec(m_1hr_opt, 1800)

optimize!(m_1hr_opt)

# Print total profit
println("Total profit: ", objective_value(m_1hr_opt))

# Specific movement of trucks
for t in 1:T_1hr, i in 1:N, j in 1:N 
    if value(x[i,j,t]) > 0.9999
    println("Truck moved from ", i, " to ", j, " at time ", t)
    end
end

In [11]:
### Optimization model for 4 hour, conservative approximation

# Create the model
m_4hr_cons = Model()

# Declare all the decision variables
@variable(m_4hr_cons, x[1:N,1:N,1:T_4hr] >= 0, Int)

@variable(m_4hr_cons, y[1:K], Bin)

# Define the objective using expression
@expression(m_4hr_cons, objective_expr,
    sum(R[k]y[k] for k in 1:K) - sum(C[i,j]x[i,j,t] for i in 1:N, j in 1:N, t in 1:T_4hr-Tau_4hr_cons[i,j]))

@objective(m_4hr_cons, Max, objective_expr)

# Write all the constraints
@constraint(m_4hr_cons, trucks_flow[i in 1:N, t in 1:T_4hr-1],
        A_4hr_cons[i,t] + sum(x[j,i,t-Tau_4hr_cons[j,i]] for j in 1:N if t-Tau_4hr_cons[j,i] >= 1) == sum(x[i,j,t] for j in 1:N))
    
@constraint(m_4hr_cons, trucks_end[i in 1:N], 
    sum(x[j,i,T_4hr-Tau_4hr_cons[j,i]] for j in 1:N if T_4hr-Tau_4hr_cons[j,i] >= 1) == B[i])

@constraint(m_4hr_cons, loads_carry[k in 1:K], 
    sum(x[O[k],Des[k],t] for t in S_4hr_cons[k]:L_4hr_cons[k]) >= y[k])

# Solve model
set_optimizer(m_4hr_cons, HiGHS.Optimizer)
set_time_limit_sec(m_4hr_cons, 1800)

optimize!(m_4hr_cons)

# Print total profit
println("Total profit: ", objective_value(m_4hr_cons))

# Specific movement of trucks
for t in 1:T_4hr, i in 1:N, j in 1:N 
    if value(x[i,j,t]) > 0.9999
    println("Truck moved from ", i, " to ", j, " at time ", t)
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
823 rows, 16401 cols, 32860 nonzeros
725 rows, 14649 cols, 29356 nonzeros
Objective function is integral with scale 5

Solving MIP model with:
   725 rows
   14649 cols (523 binary, 14126 integer, 0 implied int., 0 continuous)
   29356 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   14968           -inf                 inf        0      0      0         0     1.4s
         0       0         0   0.00%   8083.55         -inf                 inf        0      0      3      1096    71.5s
         0       0         0   0.00%   8027.32         -inf                 inf       48      3      9      1168   136.3s
         0       0         0   0.00%   8027.32         -

In [ ]:
### Optimization model for 4 hour, optimistical approximation

# Create the model
m_4hr_opt = Model()

# Declare all the decision variables
@variable(m_4hr_opt, x[1:N,1:N,1:T_4hr] >= 0, Int)

@variable(m_4hr_opt, y[1:K], Bin)

# Define the objective using expression
@expression(m_4hr_opt, objective_expr,
    sum(R[k]y[k] for k in 1:K) - sum(C[i,j]x[i,j,t] for i in 1:N, j in 1:N, t in 1:T_4hr-Tau_4hr_opt[i,j]))

@objective(m_4hr_opt, Max, objective_expr)

# Write all the constraints
@constraint(m_4hr_opt, trucks_flow[i in 1:N, t in 1:T_4hr-1],
        A_4hr_opt[i,t] + sum(x[j,i,t-Tau_4hr_opt[j,i]] for j in 1:N if t-Tau_4hr_opt[j,i] >= 1) == sum(x[i,j,t] for j in 1:N))
    
@constraint(m_4hr_opt, trucks_end[i in 1:N], 
    sum(x[j,i,T_4hr-Tau_4hr_opt[j,i]] for j in 1:N if T_4hr-Tau_4hr_opt[j,i] >= 1) == B[i])

@constraint(m_4hr_opt, loads_carry[k in 1:K], 
    sum(x[O[k],Des[k],t] for t in S_4hr_opt[k]:L_4hr_opt[k]) >= y[k])

# Solve model
set_optimizer(m_4hr_opt, HiGHS.Optimizer)
set_time_limit_sec(m_4hr_opt, 1800)

optimize!(m_4hr_opt)

# Print total profit
println("Total profit: ", objective_value(m_4hr_opt))

# Specific movement of trucks
for t in 1:T_4hr, i in 1:N, j in 1:N 
    if value(x[i,j,t]) > 0.9999
    println("Truck moved from ", i, " to ", j, " at time ", t)
    end
end

In [ ]:
### Optimization model for 15 min, conservative approximation

# Create the model
m_15min_cons = Model()

# Declare all the decision variables
@variable(m_15min_cons, x[1:N,1:N,1:T_15min] >= 0, Int)

@variable(m_15min_cons, y[1:K], Bin)

# Define the objective using expression
@expression(m_15min_cons, objective_expr,
    sum(R[k]y[k] for k in 1:K) - sum(C[i,j]x[i,j,t] for i in 1:N, j in 1:N, t in 1:T_15min-Tau_15min_cons[i,j]))

@objective(m_15min_cons, Max, objective_expr)

# Write all the constraints
@constraint(m_15min_cons, trucks_flow[i in 1:N, t in 1:T_15min-1],
        A_15min_cons[i,t] + sum(x[j,i,t-Tau_15min_cons[j,i]] for j in 1:N if t-Tau_15min_cons[j,i] >= 1) == sum(x[i,j,t] for j in 1:N))
    
@constraint(m_15min_cons, trucks_end[i in 1:N], 
    sum(x[j,i,T_15min-Tau_15min_cons[j,i]] for j in 1:N if T_15min-Tau_15min_cons[j,i] >= 1) == B[i])

@constraint(m_15min_cons, loads_carry[k in 1:K], 
    sum(x[O[k],Des[k],t] for t in S_15min_cons[k]:L_15min_cons[k]) >= y[k])

# Solve model
set_optimizer(m_15min_cons, HiGHS.Optimizer)
set_time_limit_sec(m_15min_cons, 1800)

optimize!(m_15min_cons)

# Print total profit
println("Total profit: ", objective_value(m_15min_cons))

# Specific movement of trucks
for t in 1:T_15min, i in 1:N, j in 1:N 
    if value(x[i,j,t]) > 0.9999
    println("Truck moved from ", i, " to ", j, " at time ", t)
    end
end

In [ ]:
### Optimization model for 15 min, optimistical approximation

# Create the model
m_15min_opt = Model()

# Declare all the decision variables
@variable(m_15min_opt, x[1:N,1:N,1:T_15min] >= 0, Int)

@variable(m_15min_opt, y[1:K], Bin)

# Define the objective using expression
@expression(m_15min_opt, objective_expr,
    sum(R[k]y[k] for k in 1:K) - sum(C[i,j]x[i,j,t] for i in 1:N, j in 1:N, t in 1:T_15min-Tau_15min_opt[i,j]))

@objective(m_15min_opt, Max, objective_expr)

# Write all the constraints
@constraint(m_15min_opt, trucks_flow[i in 1:N, t in 1:T_15min-1],
        A_15min_opt[i,t] + sum(x[j,i,t-Tau_15min_opt[j,i]] for j in 1:N if t-Tau_15min_opt[j,i] >= 1) == sum(x[i,j,t] for j in 1:N))
    
@constraint(m_15min_opt, trucks_end[i in 1:N], 
    sum(x[j,i,T_15min-Tau_15min_opt[j,i]] for j in 1:N if T_15min-Tau_15min_opt[j,i] >= 1) == B[i])

@constraint(m_15min_opt, loads_carry[k in 1:K], 
    sum(x[O[k],Des[k],t] for t in S_15min_opt[k]:L_15min_opt[k]) >= y[k])

# Solve model
set_optimizer(m_15min_opt, HiGHS.Optimizer)
set_time_limit_sec(m_15min_opt, 1800)

optimize!(m_15min_opt)

# Print total profit
println("Total profit: ", objective_value(m_15min_opt))

# Specific movement of trucks
for t in 1:T_15min, i in 1:N, j in 1:N 
    if value(x[i,j,t]) > 0.9999
    println("Truck moved from ", i, " to ", j, " at time ", t)
    end
end